In [1]:
import numpy as np
from bayspec.model.local import *
from bayspec.model.astro import *
from bayspec import DataUnit, Data, Infer, Plot

In [2]:
savepath = 'intermediate'

In [3]:
le = DataUnit(
    src='./LE/le2.src', 
    bkg='./LE/le2.bkg', 
    rmf='./LE/le2.rmf', 
    arf='./LE/le2.arf', 
    notc=[0.5, 4], 
    stat='cstat', 
    grpg={'min_nevt': 5})

me = DataUnit(
    src='./ME/me2.src', 
    bkg='./ME/me2.bkg', 
    rsp='./ME/me2.rsp', 
    notc=[8, 900], 
    stat='pgstat', 
    rebn={'min_nevt': 50})

data = Data([('LE', le), ('ME', me)])

data.save(savepath)
print(data)

╒════════╤════════════╤═════════════╤═════════════════╤════════╕
│  Name  │  Noticing  │  Statistic  │    Grouping     │  Time  │
╞════════╪════════════╪═════════════╪═════════════════╪════════╡
│   LE   │ [[0.5, 4]] │    cstat    │ {'min_nevt': 5} │  None  │
├────────┼────────────┼─────────────┼─────────────────┼────────┤
│   ME   │ [[8, 900]] │   pgstat    │      None       │  None  │
╘════════╧════════════╧═════════════╧═════════════════╧════════╛
╒════════╤═════════════╤═════════════╤═════════╤═════════╤══════════╕
│  par#  │  Component  │  Parameter  │  Value  │  Prior  │  Frozen  │
╞════════╪═════════════╪═════════════╪═════════╪═════════╪══════════╡
│   1    │     LE      │     sf      │    1    │  None   │   True   │
├────────┼─────────────┼─────────────┼─────────┼─────────┼──────────┤
│   2    │     LE      │     bf      │    1    │  None   │   True   │
├────────┼─────────────┼─────────────┼─────────┼─────────┼──────────┤
│   3    │     LE      │     rf      │    1    │  None 

In [4]:
tbabs_ins = AS_TbAbs()

cpl_ins = cpl()

model = tbabs_ins * cpl_ins

model.save(savepath)
print(model)

(AS_TbAbs*cpl) [add]
AS_TbAbs: Photometric absorption (Tbabs implementation), f(E) = exp(- NH * sigma(E)) contributed by Dominique Eckert
cpl: power-law model with high-energy cutoff
╒════════╤═════════════╤══════════════╤═════════╕
│  cfg#  │  Component  │  Parameter   │  Value  │
╞════════╪═════════════╪══════════════╪═════════╡
│   1    │  AS_TbAbs   │   redshift   │   0.0   │
├────────┼─────────────┼──────────────┼─────────┤
│   2    │     cpl     │   redshift   │   0.0   │
├────────┼─────────────┼──────────────┼─────────┤
│   3    │     cpl     │ pivot_energy │   1.0   │
├────────┼─────────────┼──────────────┼─────────┤
│   4    │     cpl     │   vFv_peak   │  True   │
╘════════╧═════════════╧══════════════╧═════════╛
╒════════╤═════════════╤═════════════╤═════════╤══════════════════╤══════════╕
│  par#  │  Component  │  Parameter  │  Value  │      Prior       │  Frozen  │
╞════════╪═════════════╪═════════════╪═════════╪══════════════════╪══════════╡
│   1    │  AS_TbAbs   │     N

In [5]:
infer = Infer([(data, model)])
infer.save(savepath)
print(infer)

╒════════╤═════════╤════════════════╤═════════════╤══════════════╤═════════╕
│  cfg#  │  Class  │   Expression   │  Component  │  Parameter   │  Value  │
╞════════╪═════════╪════════════════╪═════════════╪══════════════╪═════════╡
│   1    │  model  │ (AS_TbAbs*cpl) │  AS_TbAbs   │   redshift   │   0.0   │
├────────┼─────────┼────────────────┼─────────────┼──────────────┼─────────┤
│   2    │  model  │ (AS_TbAbs*cpl) │     cpl     │   redshift   │   0.0   │
├────────┼─────────┼────────────────┼─────────────┼──────────────┼─────────┤
│   3    │  model  │ (AS_TbAbs*cpl) │     cpl     │ pivot_energy │   1.0   │
├────────┼─────────┼────────────────┼─────────────┼──────────────┼─────────┤
│   4    │  model  │ (AS_TbAbs*cpl) │     cpl     │   vFv_peak   │  True   │
╘════════╧═════════╧════════════════╧═════════════╧══════════════╧═════════╛
╒════════╤═════════╤════════════════╤═════════════╤═════════════╤═════════╤══════════════════╕
│  par#  │  Class  │   Expression   │  Component  │  Param

In [6]:
post = infer.emcee(nstep=1000, discard=200, resume=True, savepath=savepath)
print(post)

100%|██████████| 1000/1000 [00:26<00:00, 37.97it/s]


╒════════╤═════════╤════════════════╤═════════════╤═════════════╤════════╤══════════╤════════╤═══════════════╤══════════════════╕
│  par#  │  Class  │   Expression   │  Component  │  Parameter  │  Mean  │  Median  │  Best  │  1sigma Best  │    1sigma CI     │
╞════════╪═════════╪════════════════╪═════════════╪═════════════╪════════╪══════════╪════════╪═══════════════╪══════════════════╡
│   1    │  model  │ (AS_TbAbs*cpl) │  AS_TbAbs   │     NH      │ 1.163  │  1.152   │ 1.096  │     1.096     │  [0.956, 1.366]  │
├────────┼─────────┼────────────────┼─────────────┼─────────────┼────────┼──────────┼────────┼───────────────┼──────────────────┤
│   2    │  model  │ (AS_TbAbs*cpl) │     cpl     │  $\alpha$   │ -1.697 │  -1.704  │ -1.662 │    -1.662     │ [-1.772, -1.619] │
├────────┼─────────┼────────────────┼─────────────┼─────────────┼────────┼──────────┼────────┼───────────────┼──────────────────┤
│   3    │  model  │ (AS_TbAbs*cpl) │     cpl     │  log$E_p$   │ 2.752  │  2.637   │ 2.27

In [7]:
fig = Plot.infer(post, style='CE')
fig.save(f'{savepath}/ctsspec')

In [8]:
fig = Plot.infer(post, style='NE')
fig.save(f'{savepath}/phtspec')

In [9]:
fig = Plot.post_corner(post, ploter='cornerpy')
fig.save(f'{savepath}/corner')

In [10]:
earr = np.logspace(np.log10(0.5), 4, 100)

modelplot = Plot.model(ploter='plotly', style='vFv', post=True)
modelplot.add_model(model, E=earr)
modelplot.add_model(cpl_ins, E=earr)
fig = modelplot.get_fig()
fig.save(f'{savepath}/model')